In [ ]:
import numpy as np
import pandas as pd
from AssetPricing.gru_scratch import CustomGRUromScratch, grad_clipping
from torch import nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch

In [ ]:
data = pd.read_csv("/Users/boningzhou/fall 2023/kaggle optiver/data/optiver-trading-at-the-close/train.csv")
# halfway_point = len(data)//10
# first_half_data = data.iloc[:halfway_point]
# first_half_data.to_csv("small_training.csv", index = False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# Load your CSV file
data = pd.read_csv('/Users/boningzhou/fall 2023/kaggle optiver/data/optiver-trading-at-the-close/train.csv')

# Filling missing values in 'far_price' and 'near_price' with 'reference_price'
data['far_price'].fillna(data['reference_price'], inplace=True)
data['near_price'].fillna(data['reference_price'], inplace=True)

# Creating new binary feature based on 'seconds_in_bucket'
data['bucket_binary'] = data['seconds_in_bucket'].apply(lambda x: 0 if x <= 290 else 1)

# Dropping unnecessary columns
data.drop(['time_id', 'row_id'], axis=1, inplace=True)

# Organizing data into sequences
data_sorted = data.sort_values(by=['stock_id', 'date_id', 'seconds_in_bucket'])
data_sorted = data_sorted.ffill()
grouped = data_sorted.groupby(['stock_id', 'date_id'])
# Selecting features and target
features = ['imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap', 'bucket_binary']
X = grouped[features].apply(lambda x: x.values.tolist())
y = grouped['target'].apply(lambda x: x.values.tolist())

original_lengths = X.apply(len)

# Determine a common sequence length
sequence_length = max(X.apply(len))

# Padding sequences to have the same length
X_padded = X.apply(lambda x: x[:sequence_length] if len(x) >= sequence_length else x + [[0]*len(features)]*(sequence_length-len(x)))
y_padded = y.apply(lambda x: x[:sequence_length] if len(x) >= sequence_length else x + [0]*(sequence_length-len(x)))

# Converting to numpy arrays
X_np = np.array(X_padded.tolist())
y_np = np.array(y_padded.tolist())

original_lengths_np = np.array(original_lengths.tolist())

# Save the processed data
np.save('X_processed.npy', X_np)
np.save('y_processed.npy', y_np)
np.save("original_lengths_np.npy", original_lengths_np)

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split

# Load the processed data (replace 'path_to_X.npy' and 'path_to_y.npy' with your file paths)
X_np = np.load('X_processed.npy')
y_np = np.load('y_processed.npy')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.2, random_state=42)

# Converting to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Creating DataLoaders
batch_size = 64  # You can adjust the batch size as needed
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
print("Input NaNs:", torch.isnan(X_train_tensor).any())
print("Target NaNs:", torch.isnan(y_train_tensor).any())


In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam
from torch.nn import L1Loss
from tqdm import tqdm

# Parameters for the GRU model
input_size = 12  # Number of input features
inner_structure = []  # Structure of the inner layers
hidden_size = 512  # Size of the hidden layer in GRU
output_size = 1  # Number of output features (depends on your task)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiate the model
model = CustomGRUromScratch(input_size, inner_structure, hidden_size, output_size, device)

# Loss Function and Optimizer
loss_fn = L1Loss()  # Replace with the appropriate loss function for your task
optimizer = Adam(model.params, lr=0.001)  # Learning rate can be adjusted

# Training Loop
num_epochs = 500  # Number of epochs
for epoch in range(num_epochs):
    total_loss = 0
    # model.train()
    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch")
    for X, y in train_loader_tqdm:
        X, y = X.to(device), y.to(device)
        state = model.begin_state(X.shape[0], device)
        y_hat, _ = model(X, state)
        # print("y_hat shape is:",y_hat.shape)
        # print("y shape is:", y.shape)
        y_hat_flat = y_hat.view(-1)
        y_flat = y.view(-1)
        
        # Compute loss
        loss = loss_fn(y_hat_flat, y_flat)
        total_loss += loss.item()
        # print(loss)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()

        # max_grad = max(param.grad.abs().max() for param in model.params if param.grad is not None)
        # print(f"Max gradient_before_clipping: {max_grad}")

        grad_clipping(model, 1)
        # max_grad = max(param.grad.abs().max() for param in model.params if param.grad is not None)
        # print(f"Max gradient after clipping: {max_grad}")

        optimizer.step()
    avg_train_loss = total_loss/ len(train_loader)
    print(f"Epoch {epoch+1}, Avg. training Loss: {avg_train_loss}")

    # Validation step (if you have validation data)
    # model.eval()
    with torch.no_grad():
        total_val_loss = 0
        for X_val, y_val in test_loader:
            X_val, y_val = X_val.to(device), y_val.to(device)
            val_state = model.begin_state(X_val.shape[0], device)
            y_val_hat, _ = model(X_val, val_state)
            y_val_hat_flat = y_val_hat.view(-1)
            y_val_flat = y_val.view(-1)
            val_loss = loss_fn(y_val_hat_flat, y_val_flat)
            total_val_loss += val_loss.item()
        
        avg_val_loss = total_val_loss / len(test_loader)
        print(f"Epoch {epoch+1}, Avg. Validation Loss: {avg_val_loss}")

In [ ]:
import torch.nn as nn

class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x is a regular tensor, not a packed sequence
        lstm_out, _ = self.lstm(x)
        # Take the output of the last time step for each sequence
        last_time_step = lstm_out[:, -1, :]
        y_pred = self.linear(last_time_step)
        return y_pred


input_size = 12  # Number of features in input
hidden_size = 128  # Number of features in hidden state
output_size = 1  # Output size
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
import torch
from torch import nn
from torch.optim import Adam

# Assuming SimpleLSTM class is already defined as per previous instructions

# Initialize the model
model = SimpleLSTM(input_size, hidden_size, output_size)
model.to(device)  # Move model to the device (CPU or GPU)

# Loss and optimizer
loss_fn = nn.L1Loss()  # MAE Loss
optimizer = Adam(model.parameters(), lr=0.001)

# Gradient clipping value
clip_value = 1

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # Forward pass
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        # print(loss)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        optimizer.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}')

    # Validation step
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(test_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}')

print("Training and evaluation complete!")
